# Computation Theory

In [1]:
#imports

import numpy as np

## Symbol Notation
---
| Symbol | Name | Operation Description | Example `(x = 0b1100, y = 0b1010)`| 
|:-:|:-:|:-:|:-----------------:|
| & | Bitwise AND | Compares each bit of two numbers. Bit is *1* only if both bits are *1*. | `x & y = 0b1000` | 
| ` | Bitwise OR |Compares each bit of two numbers. Bit is *1* if either bit is *1*. | ``x ` y = 0b1110`` | 
| ^ | Bitwise XOR | Bit is 1 if the two bits are *different*. | `x ^ y = 0b0110` | 
| ~ | Bitwise NOT | Flips every bit *(1 > 0, 0 > 1)*. Works as *-(x+1)* in Python due to two’s complement. | `~x = -0b1101` |
| << | Left Shift | Shifts bits to the left by *n* places. Fills in zeros on the right. | `x << 2 = 0b110000` | 
| >> | Right Shift | Shift bits to the right by *n* places. Fills in the zeros on the left. | `x >> 2 = 0b0011` |  

# Problem 1: Binary Words and Operations

---

## Question To Solve

Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the **Secure Hash Standard** (see page 10).

### Required Functions:

| Function | Standard Notation | Description |
|:---------|:------------------|:------------|
| `Parity(x, y, z)` | - | XOR-based parity function |
| `Ch(x, y, z)` | - | Choose function |
| `Maj(x, y, z)` | - | Majority function |
| `Sigma0(x)` | Σ₀²⁵⁶(x) | Upper-case Sigma 0 (rotations: 2, 13, 22) |
| `Sigma1(x)` | Σ₁²⁵⁶(x) | Upper-case Sigma 1 (rotations: 6, 11, 25) |
| `sigma0(x)` | σ₀²⁵⁶(x) | Lower-case sigma 0 (rotations: 7, 18; shift: 3) |
| `sigma1(x)` | σ₁²⁵⁶(x) | Lower-case sigma 1 (rotations: 17, 19; shift: 10) |

### Requirements:

1. Document each function with a clear **docstring**  
1. Explain its **purpose and behavior** in Markdown  
1. Test it with **appropriate examples** to verify correctness  
1. Use **32-bit unsigned integers** (`np.uint32`) throughout

---

## Parity Function Documentation

---

### Overview
The **parity function** is one of the logical functions used in the SHA-1 algorithm. It takes in three 32-bit words and compares their bits to decide the result for each position. Parity looks at each bit of the three inputs and checks if an **odd number of them are 1**. If the number of 1s is odd, the result bit is 1; if it's even, the result bit is 0.

### How It Works
The parity function works by applying the **XOR operator** to each bit of the three inputs (`x`, `y`, and `z`). XOR returns `1` if the number of `1` bits is **odd**, and `0` if its number of bits is **even**.

For exmaple, if we look at a single bit position across `x`, `y`, and `z`:

| x | y | z | Result (x ⊕ y ⊕ z) |
|:-:|:-:|:-:|:-----------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 |

You can see from the table that the result is `1` whenever **an odd number of inputs are 1**. 

In Python, this can be done simply by XORing the three values:
``x ^ y ^ z``

When using 32-bit unsigned integers, this operation is applied to **all 32 bits at once**, producing a new 32-bit result.

You can also conclude from the table that the order of either the `1` or `0` bits is not significant in the outcome. This means that the parity function is **commutative**.

### Why It's Used

The parity function is used in **specific rounds of the SHA-1 algorithm** to combine three different 32-bit values in a balanced way. XOR is useful because it dosen't favour any single input - the output changes if **any one** of the inputs changes.

This properly helps SHA-1 achieve **good diffusion**, meaning small changes in the input data quickly spread through the algorithm and affect many bits of the final hash.

Parity is also very simple to calculate, which makes it efficient to use repeatedly during the hashing process without slowing things down.

### Function Signature

Below is the Python implementation of the **parity function**. It takes three 32-bit integers (`x`, `y`, and `z`) and returns their bitwise XOR as a 32-bit unsigned integer. This matches the behavior defined in the SHA-1 standard:

```python
import numpy as np

def Parity(x: int, y: int, z: int) -> np.uint32:

In [2]:
def Parity(x,y,z):
    """
    Computes the parity (bitwise XOR) of three 32-bit unsigned integers.

    This function takes three integer inputs (x, y, z), converts them into
    32-bit unsigned integers to match the behavior expected in the SHA-1
    algorithm, and then applies the XOR operation across all 32 bits of
    each input. The result is a single 32-bit unsigned integer where each
    bit represents the parity of the corresponding bits of x, y, and z.

    The parity is 1 if an odd number of the three bits are 1, and 0 if an
    even number of the three bits are 1. This operation is commutative
    and matches the definition used in the Secure Hash Standard (FIPS 180-4).

    Args:
        x (int): First integer input.
        y (int): Second integer input.
        z (int): Third integer input.

    Returns:
        np.uint32: The bitwise XOR (parity) of x, y, and z as a 32-bit unsigned integer.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32(x ^ y ^ z)

### Testing the Parity Function

To make sure the `Parity` function works correctly, we can test it with a few simple inputs where we can easily calculate the expected result by hand.

For example:

- If all three inputs are `0`, the result should be `0` because no bits are set.  
- If one input is `1` and the others are `0`, the result should be `1`.  
- If two inputs are `1`, the result should be `0` (because 2 is even).  
- If all three inputs are `1`, the result should be `1` (because 3 is odd).

### Parity Function Test Cases:
---

In [5]:
test_cases = [
    (0, 0, 0),  # Expected: 0
    (1, 0, 0),  # Expected: 1
    (1, 1, 0),  # Expected: 0
    (1, 1, 1),  # Expected: 1
    (0b1010, 0b1100, 0b0110),  # Expected 0
    (0xFFFFFFFF, 0x00000000, 0xFFFFFFFF),  # Expected: 0 
]

for x, y, z in test_cases:
    result = Parity(x,y,z)
    print(f"Parity({x}, {y}, {z}) = {result:04b}")

Parity(0, 0, 0) = 0000
Parity(1, 0, 0) = 0001
Parity(1, 1, 0) = 0000
Parity(1, 1, 1) = 0001
Parity(10, 12, 6) = 0000
Parity(4294967295, 0, 4294967295) = 0000


### Expected Results From Parity Function Tests

- `Parity(0, 0, 0)` → `0`  
- `Parity(1, 0, 0)` → `1`  
- `Parity(1, 1, 0)` → `0`  
- `Parity(1, 1, 1)` → `1`  
- `Parity(0b1010, 0b1100, 0b0110)` → `0`  
- `Parity(0xFFFFFFFF, 0x00000000, 0xFFFFFFFF)` → `0`

These tests confirm that the function behaves exactly as expected, matching the bitwise XOR logic defined in the SHA-1 specification.


## Choose Function Documentation

---

### Overview
The **choose function** (often written as `Ch`) is another logical function used in SHA-1.  
It takes three 32-bit words: `x`, `y`, and `z`.  
For each bit position, it **chooses** between `y` and `z` based on the bit of `x`:

- If the bit of `x` is **1**, the result takes the bit from **`y`**.
- If the bit of `x` is **0**, the result takes the bit from **`z`**.

Because of this, people say: **“`x` chooses between `y` and `z`.”**

### How It Works

Bit-by-bit, the choose function is defined as:

`Ch(x, y, z) = (x AND y) XOR ((NOT x) AND z)`

This formula matches the “choose” idea because of how the bitwise operations work:

- When a bit of **x is 1**, the expression `(x AND y)` keeps the bit from **y** in that position  
  At the same time, `(~x AND z)` becomes `0 AND z` for that bit (because NOT 1 = 0), so the bit from **z** is ignored there.

- When a bit of **x is 0**, `(x AND y)` becomes `0 AND y` → which is always 0.  
  Meanwhile, `(~x AND z)` becomes `1 AND z` (because NOT 0 = 1), so the bit from **z** is kept in that position.

- The two parts are then joined together with XOR, but since **only one of the two expressions can produce a 1 for any bit**, XOR here is effectively just combining them without any conflict.

So, for each bit:
- **x = 1** → choose the bit from **y**.  
- **x = 0** → choose the bit from **z**.

This is why it’s called the “choose” function — `x` decides, bit by bit, whether to pick the value from `y` or from `z`.

**Example Table:**
| x | y | z | Ch(x, y, z) |
|:-:|:-:|:-:|:------------:|
| 0 | 0 | 0 |      0       |
| 0 | 0 | 1 |      1       | 
| 0 | 1 | 0 |      0       |
| 0 | 1 | 1 |      1       |
| 1 | 0 | 0 |      0       | 
| 1 | 0 | 1 |      0       |
| 1 | 1 | 0 |      1       |
| 1 | 1 | 1 |      1       |

You can see the pattern:  
- When **x=0**, the output equals **z**.  
- When **x=1**, the output equals **y**.

### Why It’s Used
The choose function helps SHA-1 **mix data in a controlled way**:
- It lets one value (`x`) **select** bits from two other values (`y` and `z`).
- A **single bit change** in `x` can flip which source is chosen at many positions, helping with **diffusion** (small input changes affect many output bits).
- It is also **cheap to compute** on hardware and in code (only AND, NOT, XOR).

### Function Signature

```python
import numpy as np

def Choose(x: int, y: int, z: int) -> np.uint32:


In [30]:
def Choose(x,y,z):
    """
    function takes in three parameters x,y,z (integers).
    Converts them to 32-bit unsigned integers.
    Chooses bits from y and z based on the bits of x.
    If a bit in x is 1, the corresponding bit from y is chosen.
    If a bit in x is 0, the corresponding bit from z is chosen.

    Args:
        x (int): first integer input
        y (int): second integer input
        z (int): third integer input

    Returns:
        32-bit unsigned integer: result of the expression (x & y) ^ (~x & y)

    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ ((~x) & z))

### Choose Function Test Case:
---

In [35]:
test_cases = [
    (0, 0, 0),  # x=0, choose z, 0
    (0, 0, 1),  # x=0, choose z, 1
    (0, 1, 0),  # x=0, choose z, 0
    (0, 1, 1),  # x=0, choose z, 1

    (1, 0, 0),  # x=1, choose y, 0
    (1, 0, 1),  # x=1, choose y, 0
    (1, 1, 0),  # x=1, choose y, 1
    (1, 1, 1),  # x=1, choose y, 1

    (0b0101, 0b1111, 0b0000),  # choose y at x=1 bits, 0b0101
    (0b1010, 0b1111, 0b0000),  # choose y at x=1 bits, 0b1010
]

for x, y, z in test_cases:
    result = Choose(x, y, z)
    print(f"x={x}, y={y}, z={z} = result={result:04b}")


x=0, y=0, z=0 = result=0000
x=0, y=0, z=1 = result=0001
x=0, y=1, z=0 = result=0000
x=0, y=1, z=1 = result=0001
x=1, y=0, z=0 = result=0000
x=1, y=0, z=1 = result=0000
x=1, y=1, z=0 = result=0001
x=1, y=1, z=1 = result=0001
x=5, y=15, z=0 = result=0101
x=10, y=15, z=0 = result=1010


### Expected Results for Choose Function Tests

- (0, 0, 0) = 0  
- (0, 0, 1) = 1  
- (0, 1, 0) = 0  
- (0, 1, 1) = 1  

- (1, 0, 0) = 0  
- (1, 0, 1) = 0  
- (1, 1, 0) = 1  
- (1, 1, 1) = 1  

- (0b0101, 0b1111, 0b0000) = 0b0101  
- (0b1010, 0b1111, 0b0000) = 0b1010  

These results match the expected behavior of the **Choose** function:
- When `x = 0`, the result equals `z`.
- When `x = 1`, the result equals `y`.
- When `x` is mixed, the function chooses bits from `y` where `x` is 1 and from `z` where `x` is 0.


## Majority Function Documentation

---

### Overview
The **majority function** (often written as `Maj`) is another logical function used in SHA-1.  
It takes three 32-bit words: `x`, `y`, and `z`.  
For each bit position, it outputs the **majority value** among the three bits:

- If **at least two of the bits are 1**, the result is **1**.  
- If **at least two of the bits are 0**, the result is **0**.

This is why it's called the **majority** function — the output reflects whichever value (0 or 1) is in the majority among `x`, `y`, and `z` for each bit.

### How It Works

Bit-by-bit, the majority function is defined as:

`Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z)`

This formula works because:
- If **at least two inputs** have a 1 at a bit position, then **at least one** of the AND combinations will be 1, and the XOR of the three terms will produce 1.
- If fewer than two bits are 1, all AND combinations will be 0, so the output will be 0.

**Example Table:**

| x | y | z | Maj(x, y, z) |
|:-:|:-:|:-:|:------------:|
| 0 | 0 | 0 |      0       |
| 0 | 0 | 1 |      0       |
| 0 | 1 | 0 |      0       |
| 0 | 1 | 1 |      1       |
| 1 | 0 | 0 |      0       |
| 1 | 0 | 1 |      1       |
| 1 | 1 | 0 |      1       |
| 1 | 1 | 1 |      1       |

You can see the pattern:  
- When **at least two bits are 1**, the output is 1.  
- When fewer than two bits are 1, the output is 0.

### Why It’s Used
The majority function helps SHA-1 **mix data** in a way that depends on **the agreement between bits** of three different words.  
It is used in certain rounds of SHA-1 to introduce **non-linearity** and improve **diffusion**.  
It’s also efficient to compute on both hardware and software because it only uses AND and XOR operations.

### Function Signature

```python
import numpy as np

def Majority(x: int, y: int, z: int) -> np.uint32:


In [16]:
def Majority(x,y,z):
    """
    function takes in three parameters x,y,z (integers).
    Converts them to 32-bit unsigned integers.
    Returns the majority value among the three inputs.

    Args:
        x (int): first integer input
        y (int): second integer input
        z (int): third integer input

    Returns:
        32-bit unsigned integer: result of the expression (x & y) ^ (x & z) ^ (y & z) 
        
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ (x & z) ^ (y & z))

### Majority Function Test Cases
---

In [52]:
test_cases = [
    (0, 0, 0),  # all zeros → majority 0
    (0, 0, 1),  # two 0s, one 1 = majority 0
    (0, 1, 0),  # two 0s, one 1 = majority 0
    (0, 1, 1),  # two 1s, one 0 = majority 1
    (1, 0, 0),  # two 0s, one 1 = majority 0
    (1, 0, 1),  # two 1s, one 0 = majority 1
    (1, 1, 0),  # two 1s, one 0 = majority 1
    (1, 1, 1),  # all 1s = majority 1

    # multi-bit cases
    (0b0101, 0b1111, 0b0000),  # check bit-by-bit majority
    (0b1010, 0b1111, 0b0000),
]

for x, y, z in test_cases:
    result = Majority(x, y, z)
    print(f"x={x}, y={y}, z={z}  => {result:04b}")


x=0, y=0, z=0  => 0000
x=0, y=0, z=1  => 0000
x=0, y=1, z=0  => 0000
x=0, y=1, z=1  => 0001
x=1, y=0, z=0  => 0000
x=1, y=0, z=1  => 0001
x=1, y=1, z=0  => 0001
x=1, y=1, z=1  => 0001
x=5, y=15, z=0  => 0101
x=10, y=15, z=0  => 1010


### Expected Results for Majority Function Tests

- (0, 0, 0) = 0  
- (0, 0, 1) = 0  
- (0, 1, 0) = 0  
- (0, 1, 1) = 1  

- (1, 0, 0) = 0  
- (1, 0, 1) = 1  
- (1, 1, 0) = 1  
- (1, 1, 1) = 1  

- (0b0101, 0b1111, 0b0000) = 0b0101  
- (0b1010, 0b1111, 0b0000) = 0b1010  

These results match the expected behavior of the **Majority** function:
- If at least **two bits are 1**, the output bit is **1**.  
- If fewer than two bits are 1 (i.e., two or more 0s), the output bit is **0**.  
- For multi-bit values, this rule is applied **bit by bit** across all positions.


# Sigma Functions

---

### Sigma0 Function Documentation

---

### Overview
The **Sigma0** function (often written as `Σ₀`) is a logical function used in SHA-256.  
It takes a single 32-bit word `x` and performs three **bitwise right rotations**, then combines the results with XOR.  
This function is designed to **scramble bits** efficiently, helping spread input differences across the state.

The formula for Sigma0 is:
`Σ₀(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)`


Where `ROTRⁿ(x)` means rotating the bits of `x` **n positions to the right**.

### How It Works

1. Convert `x` to a 32-bit unsigned integer to ensure all operations wrap around properly.  
2. Perform three **right rotations**:
   - Rotate by 2 bits
   - Rotate by 13 bits
   - Rotate by 22 bits
3. **XOR** the three rotated results together to produce the final 32-bit output.

This mixing operation ensures that small changes in `x` will affect many output bits, improving **diffusion** in the SHA-256 algorithm.

### Why We Use It

The Sigma0 function is used in the **SHA-256 compression step** to ensure that the bits of the working variables are **well-diffused**.  
This means:
- A **small change in the input** will cause a **large, unpredictable change** in the output.  
- It introduces **non-linearity** without losing information, since rotations keep all bits.  
- It strengthens the hash function against **collision** and **preimage** attacks by increasing bit interdependence.  

Sigma0, together with Sigma1, plays a key role in **updating the message schedule and state** during each round of SHA-256.

### Function Signature

```python
import numpy as np

def Sigma0(x: np.uint32):

In [2]:
def Sigma0(x):
    """
    function takes in one paramater (integer x)
    Converts x to a 32-bit unsigned integer
    returns Big Sigma O by using its formula

    Args: 
        x (int): only int value

    Return:
        a 32-bit unsigned integer: result of (ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x))

    """
    x = np.uint32(x)
    return np.uint32(
        ((x >> 2) | (x << (32 - 2))) ^
        ((x >> 13) | (x << (32 - 13))) ^
        ((x >> 22) | (x << (32 - 22)))
    )


## Sigma0 Test Cases

- - - 

In [14]:
# Test cases for Sigma0(x)
# Formula: Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x)

test_cases = [
    0x00000000,
    0xFFFFFFFF,
    0x6A09E667,
    0x12345678,
    0x80000000,
    0x00000001,
]

for x in test_cases:
    result = Sigma0(x)
    print(f"x={x:08X}  ({x:032b})  =>  Σ0(x)={result:08X}  ({result:032b})")


x=00000000  (00000000000000000000000000000000)  =>  Σ0(x)=00000000  (00000000000000000000000000000000)
x=FFFFFFFF  (11111111111111111111111111111111)  =>  Σ0(x)=FFFFFFFF  (11111111111111111111111111111111)
x=6A09E667  (01101010000010011110011001100111)  =>  Σ0(x)=CE20B47E  (11001110001000001011010001111110)
x=12345678  (00010010001101000101011001111000)  =>  Σ0(x)=66146474  (01100110000101000110010001110100)
x=80000000  (10000000000000000000000000000000)  =>  Σ0(x)=20040200  (00100000000001000000001000000000)
x=00000001  (00000000000000000000000000000001)  =>  Σ0(x)=40080400  (01000000000010000000010000000000)


### Expected Results for Sigma0(x)

- (0x00000000) = 0x00000000  
- (0xFFFFFFFF) = 0xFFFFFFFF  
- (0x6A09E667) = 0xCE20B47E 
- (0x12345678) = 0x66146474
- (0x80000000) = 0x20040200
- (0x00000001) = 0x40080400   

These results match the expected behavior of the **Sigma0** function:  
- It performs bitwise **right rotations** of `x` by 2, 13, and 22 bits.  
- The rotated values are then combined using **XOR** operations.  
- This produces the SHA-256 **Big Sigma 0** output used in message schedule and compression steps.


## Sigma1 Function Documentaion

---

### Overview
The **Sigma1** function (often written as `Σ₁`) is a logical function used in SHA-256.  
It takes a single 32-bit word `x` and performs three **bitwise right rotations**, then combines the results with XOR.  
This operation helps to **mix the bits** of the word and is part of the message schedule and compression functions in SHA-256.

The formula for Sigma1 is:
`Σ₁(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)`


Where `ROTRⁿ(x)` means rotating the bits of `x` **n positions to the right**.

### How It Works

1. **Convert** `x` to a 32-bit unsigned integer to ensure proper bit operations.  
2. Perform three **right rotations**:
   - Rotate by 6 bits
   - Rotate by 11 bits
   - Rotate by 25 bits
3. **XOR** the three rotated values together to produce the final 32-bit result.

This function ensures that **no information is lost**, since rotations wrap bits around, and the XOR spreads bit changes across multiple positions, improving **diffusion**.

### Why We Use It

The Sigma1 function is used in **SHA-256** to help mix up the bits of a value in a way that’s hard to predict.  
- It makes sure that **even a tiny change** in the input causes a **big change** in the output.  
- Rotating by different amounts and XORing helps **spread bits around**, while keeping all the original information.  
- This improves how well the algorithm **mixes data** and makes it **harder to break**.  
- Sigma1 is used in each round to **update the working variables**, along with Sigma0.

### Function Signature

```python
import numpy as np

def Sigma1(x: np.uint32):

In [17]:
def Sigma1(x):
    """
    function takes in one parameter (integer x)
    Converts x to a 32-bit unsigned integer
    returns Big Sigma 1 by using its formula

    Args: 
        x (int): only int value

    Return:
        a 32-bit unsigned integer: result of (ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x))

    """
    x = np.uint32(x)

    return np.uint32(
    ((x >> 6) | (x << (32 - 6))) ^
    ((x >> 11) | (x << (32 - 11))) ^
    ((x >> 25) | (x << (32 - 25)))
    )

## Sigma1 Test Cases

- - - 

In [19]:
# Test cases for Sigma1(x)
# Formula: Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)

test_cases = [
    0x00000000,
    0xFFFFFFFF,
    0x6A09E667,
    0x12345678,
    0x80000000,
    0x00000001,
]

for x in test_cases:
    result = Sigma1(x)
    print(f"x={x:08X}  ({x:032b})  =>  Σ1(x)={result:08X}  ({result:032b})")


x=00000000  (00000000000000000000000000000000)  =>  Σ1(x)=00000000  (00000000000000000000000000000000)
x=FFFFFFFF  (11111111111111111111111111111111)  =>  Σ1(x)=FFFFFFFF  (11111111111111111111111111111111)
x=6A09E667  (01101010000010011110011001100111)  =>  Σ1(x)=55B65510  (01010101101101100101010100010000)
x=12345678  (00010010001101000101011001111000)  =>  Σ1(x)=3561ABDA  (00110101011000011010101111011010)
x=80000000  (10000000000000000000000000000000)  =>  Σ1(x)=02100040  (00000010000100000000000001000000)
x=00000001  (00000000000000000000000000000001)  =>  Σ1(x)=04200080  (00000100001000000000000010000000)


### Expected Results for Sigma1(x)

- (0x00000000) = 0x00000000  
- (0xFFFFFFFF) = 0xFFFFFFFF  
- (0x6A09E667) = 0x55B65510  
- (0x12345678) = 0x3561ABDA  
- (0x80000000) = 0x02100040  
- (0x00000001) = 0x04200080  

These results match the expected behavior of the **Sigma1** function:  
- It performs bitwise right rotations of `x` by 6, 11, and 25 bits.  
- The rotated values are combined using XOR.  
- All arithmetic is on 32-bit unsigned words.


## sigma0 Function Documentation

---

### Overview
The **sigma0** function (often written as `σ₀`) is a logical function used in **SHA-256**.  
It takes a single 32-bit word `x` and applies a combination of **two right rotations** and **one right shift**, then XORs the results together.  
This function is part of the **message schedule** in SHA-256 and helps to mix bits in a way that’s both efficient and unpredictable.

The formula for sigma0 is:
`σ₀(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)`


Where:
- `ROTRⁿ(x)` means rotating the bits of `x` **n positions to the right** (bits wrap around).  
- `SHRⁿ(x)` means shifting the bits of `x` **n positions to the right**, with zeros filled on the left.

### How It Works

1. Convert `x` to a 32-bit unsigned integer to keep all operations within 32 bits.  
2. Perform two **right rotations**:
   - Rotate by 7 bits
   - Rotate by 18 bits
3. Perform a **right shift** by 3 bits (bits are shifted out and replaced with zeros).  
4. XOR the three results together to get the final value.

This combination of rotations and a shift ensures that the bits are **rearranged and mixed**, while also introducing some **asymmetry** (because shifting loses information, unlike rotating).

### Why We Use It

The sigma0 function is used during the **message schedule expansion** in SHA-256.  
- It helps spread the influence of each bit of the message across many positions.  
- The mix of rotations and a shift makes the output **less predictable** and improves **diffusion**, which strengthens security.  
- Sigma0 ensures that even small changes in the input will affect multiple bits in the expanded message, helping to achieve the **avalanche effect** early in the hashing process.

### Function Signature

```python
import numpy as np

def sigma0(x: np.uint32):

In [20]:
def sigma0(x):
    """
    function takes in one parameter (integer x)
    Converts x to a 32-bit unsigned integer
    returns Small Sigma 0 by using its formula

    Args: 
        x (int): only int value

    Return:
        a 32-bit unsigned integer: result of (ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x))

    """
    x = np.uint32(x)

    return np.uint32(
        ((x >> 7) | x << ((32 - 7))) ^
        ((x >> 18) | x << ((32 - 18))) ^
        ((x >> 3))
    )

## sigma0 Test Cases

- - -

In [22]:
# Test cases for sigma0(x)
# σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)

test_cases = [
    0x00000000,
    0xFFFFFFFF,
    0x6A09E667,
    0x12345678,
    0x80000000,
    0x00000001,
]

for x in test_cases:
    result = sigma0(x)
    print(f"x={x:08X}  ({x:032b})  =>  σ0(x)={result:08X}  ({result:032b})")


x=00000000  (00000000000000000000000000000000)  =>  σ0(x)=00000000  (00000000000000000000000000000000)
x=FFFFFFFF  (11111111111111111111111111111111)  =>  σ0(x)=1FFFFFFF  (00011111111111111111111111111111)
x=6A09E667  (01101010000010011110011001100111)  =>  σ0(x)=BA0CF582  (10111010000011001111010110000010)
x=12345678  (00010010001101000101011001111000)  =>  σ0(x)=E7FCE6EE  (11100111111111001110011011101110)
x=80000000  (10000000000000000000000000000000)  =>  σ0(x)=11002000  (00010001000000000010000000000000)
x=00000001  (00000000000000000000000000000001)  =>  σ0(x)=02004000  (00000010000000000100000000000000)


### Expected Results for sigma0(x)

- (0x00000000) = 0x00000000  
- (0xFFFFFFFF) = 0x1FFFFFFF  
- (0x6A09E667) = 0xBA0CF582  
- (0x12345678) = 0xE7FCE6EE  
- (0x80000000) = 0x11002000  
- (0x00000001) = 0x02004000  

These results match the expected behavior of the **sigma0** function:
- It performs right rotations by 7 and 18 bits, and a logical right shift by 3.
- Operates on 32-bit unsigned integers.


## sigma1 Function Documentation

---

### Overview
The **sigma1** function (often written as `σ₁`) is a logical function used in **SHA-256**.  
It takes a single 32-bit word `x` and applies a combination of **two right rotations** and **one right shift**, then XORs the results together.  
This function is part of the **message schedule expansion** and helps spread the influence of the input bits in a way that improves security.

The formula for sigma1 is:
`σ₁(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)`


Where:
- `ROTRⁿ(x)` rotates the bits of `x` **n positions to the right** (bits wrap around).  
- `SHRⁿ(x)` shifts the bits of `x` **n positions to the right**, filling in zeros on the left.

### How It Works

1. Convert `x` to a 32-bit unsigned integer to keep all operations within 32 bits.  
2. Perform two **right rotations**:
   - Rotate by 17 bits
   - Rotate by 19 bits
3. Perform a **right shift** by 10 bits (bits shifted out are discarded, zeros fill in from the left).  
4. XOR the three results together to get the final 32-bit output.

This combination allows sigma1 to **rearrange and mix bits** effectively, using both rotations (which keep all information) and a shift (which loses some bits) to make the transformation less predictable.

### Why We Use It

The sigma1 function is used during the **message schedule** stage of SHA-256 to expand the original message into many derived words.  
- It helps ensure that each original message bit affects **many future computations**.  
- Rotations and shifts together provide **non-linearity** and **diffusion**, making the hash much harder to reverse or predict.  
- It supports the **avalanche effect**, where a tiny change in the input causes widespread changes in the output.

### Function Signature

```python
import numpy as np

def sigma1(x: np.uint32):


In [24]:
def sigma1(x):
    """

    function takes in one parameter (integer x)
    Converts x to a 32-bit unsigned integer
    returns Small Sigma 1 by using its formula

    Args:
        x (int): only int value

    Return:
        a 32-bit unsigned integer: result of (ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x))
    """
    
    x = np.uint32(x)
    return np.uint32(
        ((x >> 17) | (x << (32 - 17))) ^
        ((x >> 19) | (x << (32 - 19))) ^
        (x >> 10)
    )

## sigma1 Test Cases

- - - 

In [ ]:
# Test cases for sigma1(x)
# σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)

test_cases = [
    0x00000000,
    0xFFFFFFFF,
    0x6A09E667,
    0x12345678,
    0x80000000,
    0x00000001,
]

for x in test_cases:
    result = sigma1(x)
    print(f"x={x:08X}  ({x:032b})  =>  σ1(x)={result:08X}  ({result:032b})")


x=00000000  (00000000000000000000000000000000)  =>  σ1(x)=00000000  (00000000000000000000000000000000)
x=FFFFFFFF  (11111111111111111111111111111111)  =>  σ1(x)=003FFFFF  (00000000001111111111111111111111)
x=6A09E667  (01101010000010011110011001100111)  =>  σ1(x)=CFE5DA3C  (11001111111001011101101000111100)
x=12345678  (00010010001101000101011001111000)  =>  σ1(x)=A1F78649  (10100001111101111000011001001001)
x=80000000  (10000000000000000000000000000000)  =>  σ1(x)=00205000  (00000000001000000101000000000000)
x=00000001  (00000000000000000000000000000001)  =>  σ1(x)=0000A000  (00000000000000001010000000000000)


### Expected Results for sigma1(x)

- (0x00000000) = 0x00000000  
- (0xFFFFFFFF) = 0x003FFFFF  
- (0x6A09E667) = 0xCFE5DA3C  
- (0x12345678) = 0xA1F78649  
- (0x80000000) = 0x00205000  
- (0x00000001) = 0x0000A000  

These results match the expected behavior of the **sigma1** function:  
- It performs right rotations by 17 and 19 bits, and a logical right shift by 10.  
- Operates on 32-bit unsigned integers.  


# Problem 2
- - -
## Question To Solve: 

Use **numpy** to calculate the constants listed at the bottom of page 11 of the **Secure Hash Standard**, following the steps below. These are the first `32 bits` of the fractional parts of the cube roots of the first 64 prime numbers.

1. Write a function called ``primes(n)`` that generates the **first n prime numbers**.

1. Use the function to **calculate the cube root of the first** ``64 primes``.

1. For each **cube root, extract the first ``32 bits`` of the fractional part**.

1. Display the result in **hexadecimal**.

1. Test the results against what is in the **Secure Hash Standard**.

In [ ]:
def primes(n):
    """
    Generate the first n prime numbers.
    Args:
        n (int): The number of prime numbers to generate.
    Returns:
        np.ndarray: An array containing the first n prime numbers.
    """
    primes_list = []
    num = 2
    
    while len(primes_list) < n:
        is_prime = True
        for p in primes_list:
            if num % p == 0:
                is_prime = False
                break
        
        if is_prime:
            primes_list.append(num)
        
        num += 1
            
    return np.array(primes_list)

## Calculate Cube Roots

This section computes the cube roots of the first *n* prime numbers using a generator function.

```python
for crt in cuberoot(primes(64)):
    print(crt)


In [23]:
def cuberoot(primes_list):
    """
    Calculate the cube root of each prime number in the list.
    Args:
        primes_list (list or array-like): A list or array of prime numbers.
    Returns:
        generator: A generator yielding the cube root of each prime number.
    """
    for p in primes_list:
        yield p ** (1/3)

In [25]:
for crt in cuberoot(primes(64)):
    print(crt)

1.2599210498948732
1.4422495703074083
1.7099759466766968
1.912931182772389
2.2239800905693152
2.3513346877207573
2.571281590658235
2.668401648721945
2.8438669798515654
3.072316825685847
3.1413806523913927
3.332221851645953
3.4482172403827303
3.503398060386724
3.6088260801386944
3.756285754221072
3.8929964158732604
3.936497183102173
4.0615481004456795
4.140817749422853
4.179339196381232
4.290840427026207
4.362070671454838
4.464745095584537
4.594700892207039
4.657009507803835
4.687548147653597
4.7474593985234
4.776856181035017
4.834588127111639
5.026525695313479
5.0787530781327
5.155136735475772
5.180101467380292
5.301459192380904
5.325074021614986
5.394690712109591
5.462555571281397
5.506878446387352
5.5720546555426225
5.635740794544236
5.65665282582291
5.758965220492401
5.778996565152129
5.818647867496961
5.838272460814002
5.953341813139051
6.064126994506963
6.100170200393062
6.11803317263662
6.153449493663682
6.205821794895751
6.223084253206058
6.307993548663267
6.3578611797342
6.4069

## Fractorial Extraction of Cube Roots

Writing a function that extracts the first ``32`` bits of each fractorial part of the cuberoots that were generated by the first **n** prime numbers. 

In [27]:
def frac_extrac(cuberoot_list):
    """
    Extract the first 32 bits of the fractional part of the cube roots
    of the first n prime numbers.
    Args:
        n (int): The number of prime numbers to consider.
    Returns:
        list: A list containing the first 32 bits of the fractional part
              of the cube roots of the first n prime numbers.
    """
    frac_bits = []
    
    for crt in cuberoot_list:
        # cast crt to int to get the integer part, subtract from crt to get frac part
        frac_part = crt - int(crt)
        # multiply frac part by 2^32 and cast to int to get first 32 bits
        frac_32bits = int(frac_part * (2**32))
        frac_bits.append(frac_32bits)
    
    return frac_bits

In [28]:
frac_extrac(cuberoot(primes(64)))

[1116352408,
 1899447441,
 3049323471,
 3921009573,
 961987163,
 1508970993,
 2453635748,
 2870763221,
 3624381080,
 310598401,
 607225278,
 1426881987,
 1925078388,
 2162078206,
 2614888103,
 3248222580,
 3835390401,
 4022224774,
 264347078,
 604807628,
 770255983,
 1249150122,
 1555081692,
 1996064986,
 2554220882,
 2821834349,
 2952996808,
 3210313671,
 3336571891,
 3584528711,
 113926993,
 338241895,
 666307205,
 773529912,
 1294757372,
 1396182291,
 1695183700,
 1986661051,
 2177026350,
 2456956037,
 2730485921,
 2820302411,
 3259730800,
 3345764771,
 3516065817,
 3600352804,
 4094571909,
 275423344,
 430227734,
 506948616,
 659060556,
 883997877,
 958139571,
 1322822218,
 1537002063,
 1747873779,
 1955562222,
 2024104815,
 2227730452,
 2361852424,
 2428436474,
 2756734187,
 3204031479,
 3329325298]